# TFM. Modelo predictivo llamadas Call Center

### Escenario

**Angel24**, empresa mallorquina con mas de 40 años en el sector, dispone de un servicio de Call Center, donde se realizan campañas telefonicas de todo tipo (*llamadas entrantes y salientes*) 

En base a la experiencia, gestionar el equipo de personas necesario para prestar un servicio adecuado en  base a unos crioteroios de calidad es dificil, sobretodo en las llamadas entrantes ya que no siempre se comportan igual.

Una de las campañas mas importantes que tiene este departamento de Call Center, es prestar atencion telefonica a los clientes de diferentes franquicias de **MRW**. Es un servicio de llamadas entrantes (*Inbound*)

Mediante el estudio de los datos historicos, se pretende dimensionar al equipo por slots para poder prestar el mejor servicio posible

### Fuentes de los datos

Los datos de los que disponemos son:
* Historial de llamadas
* Historial de envios que llegan a las oficinas


De ambos ficheros, disponemos informacion de 1 año.

### Pregunta objetivo

En base a lo descrito hasta el momento, queremos saber:

***¿Cuantas llamadas tendremos mañana de 10 a 12?*** Generalizando, saber el numero de llamadas por dia y hora en funcion de los envios que nos lleguen a las islas.

## Importacion de librerias 

In [10]:
from os import getenv
import pymssql
import pandas as pd

pd.options.display.max_columns = None

### Importacion de los datos de llamadas

In [11]:
df_calls = pd.read_csv('data/calls.csv', low_memory=False)

In [12]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186208 entries, 0 to 186207
Data columns (total 29 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          186208 non-null  int64  
 1   idTransaccion       186208 non-null  float64
 2   tInicio             97149 non-null   object 
 3   tFinal              186208 non-null  object 
 4   estado              186208 non-null  float64
 5   tProximoContacto    3 non-null       object 
 6   tReplanificacion    3701 non-null    object 
 7   nTAdmin             186208 non-null  int64  
 8   nTAdminExcedido     186208 non-null  int64  
 9   nTQ                 186208 non-null  int64  
 10  idSujeto            186208 non-null  float64
 11  sIdOriginal         186208 non-null  int64  
 12  sNombreCliente      186208 non-null  object 
 13  sApellido1Cliente   186208 non-null  object 
 14  sApellido2Cliente   0 non-null       float64
 15  sTelefonoCliente    0 non-null    

El DataFrame cargado de las llamadas tiene 28 columnas:
* idTransaccion = Identificador de la transaccion
* tInicio = Fecha hora inicio de la transaccion con agente
* tFinal = Fecha hora fin de la transaccion con agente
* estado = ...no sabemos siempre 1
* tProximoContacto = Fecha hora reprogramaciones
* tReplanificacion = Fecha hora reprogramaciones
* nTAdmin = Tiempo administrativo
* nTAdminExcedido = Tiempo administrativo exedido
* nTQ = Tiempo en cola
* idSujeto = Identificador del llamante
* sIdOriginal = Identificador del llamante
* sNombreCliente = Identificador del llamante
* sApellido1Cliente = Identificador del llamante
* sApellido2Cliente = Identificador del llamante
* sTelefonoCliente = Identificador del llamante
* idServicio = Identificador de servicio (grupo campañas)
* idCampanya = Identificador campañas
* idAgente = Identificador ragentes
* idSegmento = Segmentacion de llamadas entrantes
* sNombreSegmento = Segmentacion de llamadas entrantes
* sAtributoSegmento = Segmentacion de llamadas entrantes
* sFiltroSegmento = Segmentacion de llamadas entrantes
* idFinal = Identificador final de llamada (transaccion)
* sDescripcionFinal = Descripcion del final de llamada (transaccion)
* sFiltroFinal = Identificador + Descripcion del final de llamada (transaccion)
* tActualizado = Infor de copias de registros (info de sistemas)
* tCreacion = Fechahora inicio transaccion (entrada en sistema, no en agente) 
* nOrigenTransaccion = Entrantes siempre 2

### Importacion de los datos de envios por oficinas

In [52]:
# df4800 = pd.read_csv('data/mrw4800.txt', sep='\t', low_memory=False, encoding='latin-1')

In [53]:
# df4802 = pd.read_csv('data/mrw4802.txt', sep='\t', low_memory=False, encoding='latin-1')

In [54]:
# df4803 = pd.read_csv('data/mrw4803.txt', sep='\t', low_memory=False, encoding='latin-1')

In [55]:
# df4806 = pd.read_csv('data/mrw4806.txt', sep='\t', low_memory=False, encoding='latin-1')

In [56]:
# df4810 = pd.read_csv('data/mrw4810.txt', sep='\t', low_memory=False, encoding='latin-1')

In [57]:
# df4800.columns = df4802.columns

In [58]:
# df_shipping = pd.DataFrame()
# df_shipping = pd.concat([df_shipping, df4800])
# df_shipping = pd.concat([df_shipping, df4802])
# df_shipping = pd.concat([df_shipping, df4803])
# df_shipping = pd.concat([df_shipping, df4806])
# df_shipping = pd.concat([df_shipping, df4810])

In [63]:
# df_shipping.to_csv('data/shippings.csv')

In [64]:
df_shippings = pd.read_csv('data/shippings.csv', low_memory=False)

Los DataFrames cargados de los envios de las oficinas tienen 21 columnas:
* Fecha envío = Fecha creacion de envío
* Número envío = Identificador del envio
* Id. Fiscal = Identificacion fiscal cliente
* Nombre Comercial = Nombre del cliente
* Código servicio = Tipo de servicio, este esta vinculado con las horas en las que tendra que ser repartido
* Nombre Rem = Nombre remitente a la atencio de
* Población Rem = Poblacion remitente
* CP Rem = CP remitente
* Nombre vía Rem = Nombre domicilio remitente
* Nombre = Nombre del cliente destino
* Población = Poblacion destino
* Código postal = Codigo postal destino
* Nombre vía = Nombre domicilio cliente destino
* Total bultos = Numero de bultos en el envio
* Franquicia origen = Desde que oficina se envia el/los paquestes
* Franquicia destino = A que oficina se envia el/los paquestes
* Total = Importe envio
* Estado = Situacion envio
* Tipo anomalía = Incidencias en el envio
* Motivo = Motivo del estado del envio
* Importe Total = Importe total con gastos extras

In [12]:
import pandas as pd 
import operator
import re

In [13]:
def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    txt = re.sub(' +',' ', txt)
    return txt.strip().split()

In [14]:
def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

In [15]:
def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

In [30]:
#path = 'https://www.dropbox.com/s/edm5383iffurv4x/nombres.csv?dl=1'
#gender_list.to_csv("gender_list.csv")
gender_list = pd.read_csv("gender_list.csv")
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')
print(get_gender2('Margalida'))
#print(get_gender2('lopez jose maria'))
#print(get_gender2('santos contreras'))
#print(get_gender2('maria isabel lopez garcia rodriguez'))
#print(get_gender2('cami lopez'))
#print(get_gender2('majo garcia'))
#print(get_gender2('colon cristobal'))
#print(get_gender2('Cristóbal Colón'))
#print(get_gender2('Cristoforo Colombo')) 

f
